# Effective Kraus Rank

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *
from scipy.stats import gaussian_kde
from quantum_circuits import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
def evaluate_channel(channel, inputs, targets):
    kl_div = KLDiv()
    loss = kl_div(channel, inputs, targets)
    return loss

## Real Hardware

In [36]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

model1 =fit_model(channel=KrausMap(d=d, 
                                   rank=64),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  N_map=500,
                  N_spam=None,
                  num_iter_map=4000,
                  num_iter_spam=4000,
                  filename="belem_concatenate_4layer0",
                  verbose=True
                  )

  0%|          | 0/4000 [00:00<?, ?it/s]

0.0029526480899278247


  0%|          | 0/4000 [00:00<?, ?it/s]

In [37]:
np.random.seed(42)
random.seed(42)

model2 =fit_model(channel=KrausMap(d=d, 
                                   rank=64),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  N_map=500,
                  N_spam=None,
                  num_iter_map=4000,
                  num_iter_spam=4000,
                  filename="belem_concatenate_4layer1",
                  verbose=True
                  )

  0%|          | 0/4000 [00:00<?, ?it/s]

0.0028427745956133526


  0%|          | 0/4000 [00:00<?, ?it/s]

In [38]:
np.random.seed(42)
random.seed(42)

model_full = fit_model(channel=KrausMap(d=d, 
                                        rank=64),
                       spam=SPAM(init = InitialState(d),
                                 povm = CorruptionMatrix(d),
                                 optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                ),
                      N_map=500,
                      N_spam=None,
                      num_iter_map=4000,
                      num_iter_spam=4000,
                      filename="belem_concatenate_4layer2",
                      verbose = True,
                  )

  0%|          | 0/4000 [00:00<?, ?it/s]

0.0028636746735253137


  0%|          | 0/4000 [00:00<?, ?it/s]

In [54]:
channel1 = model1.channel
channel2 = model2.channel
channel_concat = channel_to_choi_map([channel1, channel2])
channel_concat.spam = channel_full.spam

channel_full = model_full.channel

In [55]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target1 = pqc_basic(n, 4).reverse_bits()
circuit_target2 = pqc_basic(n, 4).reverse_bits()

circuit_target_full = deepcopy(circuit_target1)
circuit_target_full = circuit_target_full.compose(circuit_target2)

U1 = Operator(circuit_target1).data
U2 = Operator(circuit_target2).data
U_full = Operator(circuit_target_full).data

In [56]:
channel_ideal1 = DilutedKrausMap(U1, c=0.999, d=d, rank=1)
channel_ideal2 = DilutedKrausMap(U2, c=0.999, d=d, rank=1)
channel_ideal_full = DilutedKrausMap(U_full, c=0.999, d=d, rank=1)

channel_ideal1.spam = channel1.spam
channel_ideal2.spam = channel2.spam
channel_ideal_full.spam = channel_full.spam

In [57]:
print(f" U1 vs T1: {channel_fidelity(channel_ideal1, channel1):.4f}")
print(f" U2 vs T2: {channel_fidelity(channel_ideal2, channel2):.4f}")

print(f" U Full vs T Full: {channel_fidelity(channel_ideal_full, channel_full):.4f}")
print(f" U Full vs T2*T1: {channel_fidelity(channel_ideal_full, channel_concat):.4f}")
      
print(f" T Full vs T2*T1: {channel_fidelity(channel_full, channel_concat):.4f}")

 U1 vs T1: 0.5171
 U2 vs T2: 0.5656
 U Full vs T Full: 0.4364
 U Full vs T2*T1: 0.3014
 T Full vs T2*T1: 0.6713


In [58]:
inputs_map1, targets_map1, inputs_spam1, targets_spam1 = pickle.load(
        open(f"../../data/belem_concatenate_4layer0", "rb")
    )


inputs_map2, targets_map2, inputs_spam2, targets_spam2 = pickle.load(
        open(f"../../data/belem_concatenate_4layer1", "rb")
    )

inputs_map3, targets_map3, inputs_spam3, targets_spam3 = pickle.load(
        open(f"../../data/belem_concatenate_4layer2", "rb")
    )

In [59]:
print(f"{np.abs(evaluate_channel(channel_ideal1, inputs_map1, targets_map1)):.4f}")
print(f"{np.abs(evaluate_channel(channel1, inputs_map1, targets_map1)):.4f}")

print(f"{np.abs(evaluate_channel(channel_ideal2, inputs_map2, targets_map2)):.4f}")
print(f"{np.abs(evaluate_channel(channel2, inputs_map2, targets_map2)):.4f}")

print(f"{np.abs(evaluate_channel(channel_ideal_full, inputs_map3, targets_map3)):.4f}")
print(f"{np.abs(evaluate_channel(channel_full, inputs_map3, targets_map3)):.4f}")
print(f"{np.abs(evaluate_channel(channel_concat, inputs_map3, targets_map3)):.4f}")

0.1545
0.0070
0.1194
0.0063
0.1689
0.0027
0.0274


In [61]:
channel1_slice = channel_to_choi_map([channel_full, channel2], invert_list = [False, True])
channel1_slice.spam = channel1.spam

print(f"{np.abs(evaluate_channel(channel_ideal1, inputs_map1, targets_map1)):.4f}")
print(f"{np.abs(evaluate_channel(channel1_slice, inputs_map1, targets_map1)):.4f}")
print(f"{np.abs(evaluate_channel(channel1, inputs_map1, targets_map1)):.5f}")
print(channel_fidelity(channel1, channel1_slice))

0.1545
0.0979
0.00700
tf.Tensor(1.0802189590575544, shape=(), dtype=float64)


In [62]:
channel2_slice = channel_to_choi_map([channel1, channel_full], invert_list = [True, False])
channel2_slice.spam = channel2.spam

print(f"{np.abs(evaluate_channel(channel_ideal2, inputs_map2, targets_map2)):.4f}")
print(f"{np.abs(evaluate_channel(channel2_slice, inputs_map2, targets_map2)):.4f}")
print(f"{np.abs(evaluate_channel(channel2, inputs_map2, targets_map2)):.5f}")
print(channel_fidelity(channel2_slice, channel2))

0.1194
0.0939
0.00629
tf.Tensor(0.6896768509405781, shape=(), dtype=float64)


## Ost

In [3]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

model1 =fit_model(channel=KrausMap(d=d, 
                                   rank=64),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  N_map=500,
                  N_spam=None,
                  num_iter_map=4000,
                  num_iter_spam=4000,
                  filename="belem_concatenate_8layer0",
                  verbose=True
                  )

  0%|          | 0/4000 [00:00<?, ?it/s]

0.0027129492824602593


  0%|          | 0/4000 [00:00<?, ?it/s]

In [4]:
np.random.seed(42)
random.seed(42)

model2 =fit_model(channel=KrausMap(d=d, 
                                   rank=64),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  N_map=500,
                  N_spam=None,
                  num_iter_map=4000,
                  num_iter_spam=4000,
                  filename="belem_concatenate_8layer1",
                  verbose=True
                  )

  0%|          | 0/4000 [00:00<?, ?it/s]

0.0028545437870340466


  0%|          | 0/4000 [00:00<?, ?it/s]

In [5]:
np.random.seed(42)
random.seed(42)

model_full =fit_model(channel=KrausMap(d=d, 
                                   rank=64),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  N_map=500,
                  N_spam=None,
                  num_iter_map=4000,
                  num_iter_spam=4000,
                  filename="belem_concatenate_8layer2",
                  verbose=True
                  )

  0%|          | 0/4000 [00:00<?, ?it/s]

0.003028827428731858


  0%|          | 0/4000 [00:00<?, ?it/s]

In [6]:
channel1 = model1.channel
channel2 = model2.channel
channel_full = model_full.channel

channel_concat = channel_to_choi_map([channel1, channel2])
channel_concat.spam = channel_full.spam

In [7]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target1 = pqc_basic(n, 8).reverse_bits()
circuit_target2 = pqc_basic(n, 8).reverse_bits()

circuit_target_full = deepcopy(circuit_target1)
circuit_target_full = circuit_target_full.compose(circuit_target2)

U1 = Operator(circuit_target1).data
U2 = Operator(circuit_target2).data
U_full = Operator(circuit_target_full).data

channel_ideal1 = DilutedKrausMap(U1, c=0.999, d=d, rank=1)
channel_ideal2 = DilutedKrausMap(U2, c=0.999, d=d, rank=1)
channel_ideal_full = DilutedKrausMap(U_full, c=0.999, d=d, rank=1)

channel_ideal1.spam = channel1.spam
channel_ideal2.spam = channel2.spam
channel_ideal_full.spam = channel_full.spam

In [8]:
print(f" U1 vs T1: {channel_fidelity(channel_ideal1, channel1):.4f}")
print(f" U2 vs T2: {channel_fidelity(channel_ideal2, channel2):.4f}")

print(f" U Full vs T Full: {channel_fidelity(channel_ideal_full, channel_full):.4f}")
print(f" U Full vs T2*T1: {channel_fidelity(channel_ideal_full, channel_concat):.4f}")
      
print(f" T Full vs T2*T1: {channel_fidelity(channel_full, channel_concat):.4f}")

 U1 vs T1: 0.4293
 U2 vs T2: 0.4601
 U Full vs T Full: 0.2785
 U Full vs T2*T1: 0.2006
 T Full vs T2*T1: 0.6678


In [9]:
inputs_map1, targets_map1, inputs_spam1, targets_spam1 = pickle.load(
        open(f"../../data/belem_concatenate_8layer0", "rb")
    )


inputs_map2, targets_map2, inputs_spam2, targets_spam2 = pickle.load(
        open(f"../../data/belem_concatenate_8layer1", "rb")
    )

inputs_map3, targets_map3, inputs_spam3, targets_spam3 = pickle.load(
        open(f"../../data/belem_concatenate_8layer2", "rb")
    )

In [10]:
print(f"{np.abs(evaluate_channel(channel_ideal1, inputs_map1, targets_map1)):.5f}")
print(f"{np.abs(evaluate_channel(channel1, inputs_map1, targets_map1)):.5f}")

print(f"{np.abs(evaluate_channel(channel_ideal2, inputs_map2, targets_map2)):.5f}")
print(f"{np.abs(evaluate_channel(channel2, inputs_map2, targets_map2)):.5f}")

print(f"{np.abs(evaluate_channel(channel_ideal_full, inputs_map3, targets_map3)):.5f}")
print(f"{np.abs(evaluate_channel(channel_full, inputs_map3, targets_map3)):.5f}")
print(f"{np.abs(evaluate_channel(channel_concat, inputs_map3, targets_map3)):.5f}")

0.17230
0.00193
0.15056
0.00212
0.25112
0.00003
0.01249


In [11]:
channel1_slice = channel_to_choi_map([channel_full, channel2], invert_list = [False, True])
channel1_slice.spam = channel1.spam

print(f"{np.abs(evaluate_channel(channel_ideal1, inputs_map1, targets_map1)):.4f}")
print(f"{np.abs(evaluate_channel(channel1_slice, inputs_map1, targets_map1)):.4f}")
print(f"{np.abs(evaluate_channel(channel1, inputs_map1, targets_map1)):.5f}")

0.1723
0.0924
0.00193


In [12]:
channel2_slice = channel_to_choi_map([channel1, channel_full], invert_list = [True, False])
channel2_slice.spam = channel2.spam

print(f"{np.abs(evaluate_channel(channel_ideal2, inputs_map2, targets_map2)):.4f}")
print(f"{np.abs(evaluate_channel(channel2_slice, inputs_map2, targets_map2)):.4f}")
print(f"{np.abs(evaluate_channel(channel2, inputs_map2, targets_map2)):.5f}")

0.1506
0.0592
0.00212


In [14]:
print(channel_fidelity(channel1_slice, channel1))
print(channel_fidelity(channel2_slice, channel2))

tf.Tensor(0.9289900887878287, shape=(), dtype=float64)
tf.Tensor(0.6705321557843179, shape=(), dtype=float64)
